## Harjoitustyö

## Vaihe 1

#### Tehtävä 1. Hae Digitrafficin REST-rajapintapalvelusta (JSON-formaatissa) aktiiviset junat pääteosoitteella `/live-trains` Helsingin asemalta, joiden hakuparametrit ovat seuraavat:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False


#### Tehtävä 2. Tämän jälkeen lue kyseinen data Pandasin dataframeen ilman muokkaamista, jolloin sarakkeiden otsikot ovat seuraavanlaiset:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version` 

In [2]:
import requests
import pandas as pd
import re
import json

url = 'https://rata.digitraffic.fi/api/v1/live-trains/station/HKI'
endpoint = '?arrived_trains=10&departed_trains=10&include_nonstopping=false'
new_url = url + endpoint
# req.url
req = requests.get(new_url)

# req.status_code
trainlist = req.json()

df = pd.DataFrame(trainlist)
df.head()


,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,46,2021-03-27,10,vr,IC,Long-distance,,False,False,277530168987,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'VS', 'stationUICCode': ..."
1,8118,2021-03-27,10,vr,HL,Commuter,A,False,False,277530214639,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'LPV', 'stationUICCode':..."
2,8123,2021-03-27,10,vr,HL,Commuter,A,True,False,277530238265,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,8352,2021-03-27,10,vr,HL,Commuter,E,False,False,277530155326,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'KLH', 'stationUICCode':..."
4,8355,2021-03-27,10,vr,HL,Commuter,E,True,False,277530236034,REGULAR,2020-11-06T10:17:00.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


#### Tehtävä 3. Poista kaikki muut sarakkeet paitsi `timeTableRows` ja `trainNumber`

In [3]:
for i in df:
    if i not in {'timeTableRows','trainNumber'}:
        df.drop(i, inplace=True, axis=1)
df.head()

,trainNumber,timeTableRows
0,46,"[{'stationShortCode': 'VS', 'stationUICCode': ..."
1,8118,"[{'stationShortCode': 'LPV', 'stationUICCode':..."
2,8123,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,8352,"[{'stationShortCode': 'KLH', 'stationUICCode':..."
4,8355,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


#### Tehtävä 4. Tämän jälkeen pura `timeTableRows` sarakkeen arvot jokaisesta junasta erikseen (sarakkeen arvo jokaiselle junalle sisältää useampia JSON-objekteja) ja lisää dataan mukaan junan tiedot yksilöivä `trainNumber`

In [4]:
new_df = pd.json_normalize(trainlist, record_path='timeTableRows', meta=['trainNumber']) 
new_df.head(10)

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,actualTime,differenceInMinutes,causes,trainReady.source,trainReady.accepted,trainReady.timestamp,liveEstimateTime,estimateSource,trainNumber
0,VS,288,FI,DEPARTURE,True,True,1,False,2021-03-27T08:35:00.000Z,2021-03-27T08:36:01.000Z,1.0,[],KUPLA,True,2021-03-27T08:32:09.000Z,NaN,NaN,46
1,LAI,293,FI,ARRIVAL,False,NaN,1,False,2021-03-27T08:50:00.000Z,2021-03-27T08:49:50.000Z,0.0,[],NaN,NaN,NaN,NaN,NaN,46
2,LAI,293,FI,DEPARTURE,False,NaN,1,False,2021-03-27T08:50:00.000Z,2021-03-27T08:49:50.000Z,0.0,[],NaN,NaN,NaN,NaN,NaN,46
3,TK,294,FI,ARRIVAL,True,True,1,False,2021-03-27T08:55:00.000Z,NaN,0.0,[],NaN,NaN,NaN,2021-03-27T08:55:03.000Z,LIIKE_AUTOMATIC,46
4,TK,294,FI,DEPARTURE,True,True,1,False,2021-03-27T08:56:00.000Z,NaN,0.0,[],NaN,NaN,NaN,2021-03-27T08:56:03.000Z,LIIKE_AUTOMATIC,46
5,IKY,295,FI,ARRIVAL,False,NaN,1,False,2021-03-27T09:05:00.000Z,2021-03-27T09:04:08.000Z,-1.0,[],NaN,NaN,NaN,NaN,NaN,46
6,IKY,295,FI,DEPARTURE,False,NaN,1,False,2021-03-27T09:05:00.000Z,2021-03-27T09:04:08.000Z,-1.0,[],NaN,NaN,NaN,NaN,NaN,46
7,YST,296,FI,ARRIVAL,False,NaN,,False,2021-03-27T09:10:00.000Z,NaN,-1.0,[],NaN,NaN,NaN,2021-03-27T09:09:16.000Z,LIIKE_AUTOMATIC,46
8,YST,296,FI,DEPARTURE,False,NaN,,False,2021-03-27T09:10:00.000Z,NaN,-1.0,[],NaN,NaN,NaN,2021-03-27T09:09:16.000Z,LIIKE_AUTOMATIC,46
9,SK,280,FI,ARRIVAL,True,True,1,False,2021-03-27T09:24:00.000Z,2021-03-27T09:20:44.000Z,-3.0,[],NaN,NaN,NaN,NaN,NaN,46


#### Tehtävä 5. Pudota sitten seuraavat sarakkeet datasta: `causes`, `estimateSource`, `liveEstimateTime`, `differenceInMinutes`, `commercialStop` ja `trainReady`

In [5]:
for i in new_df:
    if i in {'causes', 'estimateSource', 'liveEstimateTime', 'differenceInMinutes', 'commercialStop', 'trainReady'}:
        new_df.drop(i, inplace=True, axis=1)
new_df.tail(10)

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainReady.source,trainReady.accepted,trainReady.timestamp,trainNumber
1486,KSU,1128,FI,DEPARTURE,False,,False,2021-03-27T13:11:00.000Z,NaN,NaN,NaN,NaN,9847
1487,MLÄ,27,FI,ARRIVAL,True,4,False,2021-03-27T13:15:00.000Z,NaN,NaN,NaN,NaN,9847
1488,MLÄ,27,FI,DEPARTURE,True,4,False,2021-03-27T13:16:00.000Z,NaN,NaN,NaN,NaN,9847
1489,SIP,1254,FI,ARRIVAL,False,,False,2021-03-27T13:20:12.000Z,NaN,NaN,NaN,NaN,9847
1490,SIP,1254,FI,DEPARTURE,False,,False,2021-03-27T13:20:12.000Z,NaN,NaN,NaN,NaN,9847
1491,HNN,1164,FI,ARRIVAL,True,4,False,2021-03-27T13:25:00.000Z,NaN,NaN,NaN,NaN,9847
1492,HNN,1164,FI,DEPARTURE,True,4,False,2021-03-27T13:26:00.000Z,NaN,NaN,NaN,NaN,9847
1493,HLT,1014,FI,ARRIVAL,False,,False,2021-03-27T13:34:00.000Z,NaN,NaN,NaN,NaN,9847
1494,HLT,1014,FI,DEPARTURE,False,,False,2021-03-27T13:34:00.000Z,NaN,NaN,NaN,NaN,9847
1495,LH,100,FI,ARRIVAL,True,2,False,2021-03-27T13:40:00.000Z,NaN,NaN,NaN,NaN,9847


#### Tehtävä 6. Täytä jokaisen rivin sarakkeen `actualTime` puuttuvat arvot `scheduledTime` -sarakkeen arvolla.

In [6]:
new_df['actualTime'].fillna(new_df['scheduledTime'], inplace=True)
new_df.tail(5)

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainReady.source,trainReady.accepted,trainReady.timestamp,trainNumber
1491,HNN,1164,FI,ARRIVAL,True,4,False,2021-03-27T13:25:00.000Z,2021-03-27T13:25:00.000Z,NaN,NaN,NaN,9847
1492,HNN,1164,FI,DEPARTURE,True,4,False,2021-03-27T13:26:00.000Z,2021-03-27T13:26:00.000Z,NaN,NaN,NaN,9847
1493,HLT,1014,FI,ARRIVAL,False,,False,2021-03-27T13:34:00.000Z,2021-03-27T13:34:00.000Z,NaN,NaN,NaN,9847
1494,HLT,1014,FI,DEPARTURE,False,,False,2021-03-27T13:34:00.000Z,2021-03-27T13:34:00.000Z,NaN,NaN,NaN,9847
1495,LH,100,FI,ARRIVAL,True,2,False,2021-03-27T13:40:00.000Z,2021-03-27T13:40:00.000Z,NaN,NaN,NaN,9847


## Vaihe 2

#### Tehtävä 7. Pura tämän kansion sisällä olevan `data.html` -tiedoston `table`-elementin solujen arvot käyttäen regex-syntaksia ja yhdistä ne vaiheessa 1 luotuun dataframeen. Huom: tehtävässä tulee käyttää vain regexiä (Esim. Pandasin read_html()-metodin käyttö on kielletty)!

In [7]:
with open('data.html', 'r') as fobj:
    text = fobj.read()

# taulukon otsikot
re_pattern = r'>(.*)</'
re_search = re.findall(re_pattern, text)

# taulukon sisältö
re_pattern2 = r'td>\s*(.*)\s*<\/td'
re_search2 = re.findall(re_pattern2, text)
re_search2

# Ei niin elegantti ratkaisu, mutta onpahan tehty... kiitos vain avusta!
data = [[re_search2[0], re_search2[1], re_search2[2], re_search2[3], re_search2[4], re_search2[5], re_search2[6], re_search2[7], re_search2[8], re_search2[9]],
       [re_search2[10], re_search2[11], re_search2[12], re_search2[13], re_search2[14], re_search2[15], re_search2[16], re_search2[17], re_search2[18], re_search2[19]],
       [re_search2[20], re_search2[21], re_search2[22], re_search2[23], re_search2[24], re_search2[25], re_search2[26], re_search2[27], re_search2[28], re_search2[29]]]

df = pd.DataFrame(data, columns=re_search) 

new_df = pd.concat([new_df, df], axis=0, ignore_index=True)
new_df = new_df.replace('\t', '', regex=True)
new_df

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialTrack,cancelled,scheduledTime,actualTime,trainReady.source,trainReady.accepted,trainReady.timestamp,trainNumber
0,VS,288,FI,DEPARTURE,True,1,False,2021-03-27T08:35:00.000Z,2021-03-27T08:36:01.000Z,KUPLA,True,2021-03-27T08:32:09.000Z,46
1,LAI,293,FI,ARRIVAL,False,1,False,2021-03-27T08:50:00.000Z,2021-03-27T08:49:50.000Z,NaN,NaN,NaN,46
2,LAI,293,FI,DEPARTURE,False,1,False,2021-03-27T08:50:00.000Z,2021-03-27T08:49:50.000Z,NaN,NaN,NaN,46
3,TK,294,FI,ARRIVAL,True,1,False,2021-03-27T08:55:00.000Z,2021-03-27T08:55:00.000Z,NaN,NaN,NaN,46
4,TK,294,FI,DEPARTURE,True,1,False,2021-03-27T08:56:00.000Z,2021-03-27T08:56:00.000Z,NaN,NaN,NaN,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,HLT,1014,FI,DEPARTURE,False,,False,2021-03-27T13:34:00.000Z,2021-03-27T13:34:00.000Z,NaN,NaN,NaN,9847
1495,LH,100,FI,ARRIVAL,True,2,False,2021-03-27T13:40:00.000Z,2021-03-27T13:40:00.000Z,NaN,NaN,NaN,9847
1496,JVS,1272,FI,DEPARTURE,False,,False,2019-08-06T04:48:00.000Z,2019-08-06T04:48:00.000Z,NaN,NaN,NaN,150
1497,R702,98998,FI,ARRIVAL,False,,False,2019-08-06T06:04:39.000Z,2019-08-06T06:04:39.000Z,NaN,NaN,NaN,150


## Vaihe 3

#### Tehtävä 8. Tee SQLite tietokanta ja lisää sinne vaiheessa 2 luotu dataframe.


In [8]:
# Tätä ei saa toimimaan edelleenkään...

import sqlite3

database_connection = sqlite3.connect('vaihe3.db')
c = database_connection.cursor()

new_df.to_sql(name='train', con=database_connection, if_exists='replace', index=False)
sql_query = c.execute('''SELECT * FROM train''')
dataframe_from_sqlite = pd.read_sql(sql_query, database_connection)
dataframe_from_sqlite

database_connection.commit()
database_connection_close()

DatabaseError: Execution failed on sql '<sqlite3.Cursor object at 0x000001AF8CE43960>': operation parameter must be str